# Lesson 3: Reflection and Blogpost Writing
In this notebook, we cover the following -
* Agent reflection framework & leverage its power to create a high quality blog post
* Learn to use nested chat conversation pattern to realize a sophisticated reflection process
* Build a system where a set of reviewer agents is nested within a critic agent as inner monologue to reflect on a blog post written by a writer agent

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Embark on a journey into the world of artificial intelligence with DeepLearning.AI. Founded by Andrew Ng, this platform offers top-notch courses on deep learning, empowering learners to master the skills needed to thrive in this cutting-edge field. From neural networks to computer vision, DeepLearning.AI covers it all in an engaging and accessible manner. Whether you're a beginner or an experienced professional looking to upskill, this platform caters to all. Join the ranks of aspiring AI enthusiasts and unleash your potential with DeepLearning.AI today!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Embark on a journey into the world of artificial intelligence with DeepLearning.AI. Founded by Andrew Ng, this platform offers top-notch courses on deep learning, empowering learners to master the skills needed to thrive in this cutting-edge field. From neural networks to computer vision, DeepLearning.AI covers it all in an engaging and accessible manner. Whether you're a beginner or an experienced professional looking to upskill, this platform caters to all. Join the ranks of aspiring AI enthusiasts and unleash your potential with DeepLearning.AI today!

--------------------------------------------------------------------------------
Critic (to Writer):

This blog post effectively in

Though we see here, the writer refining its blogpost based on critque's suggestions, in many cases when we want to realize even more compact reflection workflow as inner monologue of the critic agent.

For e.g., when we want to ensure the critic agent to provide critique regarding certain aspects of work .e.g., whether the content will rank well in the search engines, whether it will attract organic traffic, whether it has ethical, legal problems etc. 

Next we see, **Nested Chat** to handle this.

**Nested chat** is effectively a chat registered as the inner monologue of an agent


## Nested chat

In [9]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [10]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [11]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [12]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [13]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [14]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

* By setting `trigger=writer`, in this nested chat, whenever the critic agent **receives** a message from the writer agent, the reflection message is automatically routed to this nested chat session for careful reflection
* After this registration step, we can now initiate the chat between the critic and the writer
**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [15]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Embark on a journey into the world of artificial intelligence with DeepLearning.AI. Founded by Andrew Ng, this platform offers top-notch courses on deep learning, empowering learners to master the skills needed to thrive in this cutting-edge field. From neural networks to computer vision, DeepLearning.AI covers it all in an engaging and accessible manner. Whether you're a beginner or an experienced professional looking to upskill, this platform caters to all. Join the ranks of aspiring AI enthusiasts and unleash your potential with DeepLearning.AI today!

--------------------------------------------------------------------------------

*************************************************

Meta Reviewer (to Critic):

Aggregated Feedback:
- SEO Reviewer suggests adding keywords, internal links, and optimizing meta tags for better search engine optimization.
- Legal Reviewer recommends verifying the use of "top-notch" and claims of "mastering skills" for accuracy and avoiding intellectual property infringements.
- Ethics Reviewer emphasizes avoiding exaggerated claims, ensuring accurate descriptions of offerings, and providing truthful evidence to support statements about the platform's effectiveness.

Final Suggestions:
1. Incorporate relevant keywords, internal links, and optimize meta tags to improve SEO.
2. Review and potentially revise language like "top-notch" and claims of skill mastery for accuracy.
3. Ensure no intellectual property infringements related to DeepLearning.AI's use.
4. Avoid exaggerated claims about benefits and outcomes of using the courses.
5. Provide accurate descriptions of the platform's offerings to avoid misleading users.
6. Support statements

## Get the summary

In [17]:
print(res.summary)

Title: Unraveling the Potential of DeepLearning.AI

Dive into the realm of artificial intelligence with DeepLearning.AI, a platform curated by Andrew Ng. Offering comprehensive courses on deep learning, it equips learners with essential skills for the AI domain. From neural networks to computer vision, DeepLearning.AI covers diverse topics accessible to all skill levels. Whether you're a novice or a seasoned professional, this platform caters to your learning needs. Embrace the opportunities that DeepLearning.AI presents to enhance your AI expertise and stay ahead in this ever-evolving field!
